# Tutorial 01: Running Sumo Simulations

This tutorial walks through the process of running non-RL traffic simulations in Flow. Simulations of this form act as non-autonomous baselines and depict the behavior of human dynamics on a network. Similar simulations may also be used to evaluate the performance of hand-designed controllers on a network. This tutorial focuses primarily on the former use case, while an example of the latter may be found in `tutorial09_controllers.ipynb`.

In this tutorial, we simulate a initially perturbed single lane ring road. We witness in simulation that as time advances the initially perturbations do not dissipate, but instead propagates and expands until vehicles are forced to periodically stop and accelerate. For more information on this behavior, we refer the reader to the following article [1].

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc. in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network.

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in `flow/networks`. In order to recreate a ring road network, we begin by importing the network `RingNetwork`.

In [1]:
from flow.networks.ring import RingNetwork

This network, as well as all other networks in Flow, is parametrized by the following arguments: 
* name
* vehicles
* net_params
* initial_config
* traffic_lights

These parameters allow a single network to be recycled for a multitude of different network settings. For example, `RingNetwork` may be used to create ring roads of variable length with a variable number of lanes and vehicles.

### 2.1 Name
The `name` argument is a string variable depicting the name of the network. This has no effect on the type of network created.

In [2]:
name = "ring_example"

### 2.2 VehicleParams
The `VehicleParams` class stores state information on all vehicles in the network. This class is used to identify the dynamical behavior of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various get methods within this class.

The initial configuration of this class describes the number of vehicles in the network at the start of every simulation, as well as the properties of these vehicles. We begin by creating an empty `VehicleParams` object.

In [3]:
from flow.core.params import VehicleParams

vehicles = VehicleParams()

Once this object is created, vehicles may be introduced using the `add` method. This method specifies the types and quantities of vehicles at the start of a simulation rollout. For a description of the various arguements associated with the `add` method, we refer the reader to the following documentation ([VehicleParams.add](https://flow.readthedocs.io/en/latest/flow.core.html?highlight=vehicleparam#flow.core.params.VehicleParams)).

When adding vehicles, their dynamical behaviors may be specified either by the simulator (default), or by user-generated models. For longitudinal (acceleration) dynamics, several prominent car-following models are implemented in Flow. For this example, the acceleration behavior of all vehicles will be defined by the Intelligent Driver Model (IDM) [2].

In [4]:
from flow.controllers.car_following_models import IDMController

Another controller we define is for the vehicle's routing behavior. For closed network where the route for any vehicle is repeated, the `ContinuousRouter` controller is used to perpetually reroute all vehicles to the initial set route.

In [5]:
from flow.controllers.routing_controllers import ContinuousRouter

Finally, we add 22 vehicles of type "human" with the above acceleration and routing behavior into the `Vehicles` class.

In [6]:
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

### 2.3 NetParams

`NetParams` are network-specific parameters used to define the shape and properties of a network. Unlike most other parameters, `NetParams` may vary drastically depending on the specific network configuration, and accordingly most of its parameters are stored in `additional_params`. In order to determine which `additional_params` variables may be needed for a specific network, we refer to the `ADDITIONAL_NET_PARAMS` variable located in the network file.

In [7]:
from flow.networks.ring import ADDITIONAL_NET_PARAMS

print(ADDITIONAL_NET_PARAMS)

{'length': 230, 'lanes': 1, 'speed_limit': 30, 'resolution': 40}


Importing the `ADDITIONAL_NET_PARAMS` dictionary from the ring road network, we see that the required parameters are:

* **length**: length of the ring road
* **lanes**: number of lanes
* **speed**: speed limit for all edges
* **resolution**: resolution of the curves on the ring. Setting this value to 1 converts the ring to a diamond.


At times, other inputs may be needed from `NetParams` to recreate proper network features/behavior. These requirements can be found in the network's documentation. For the ring road, no attributes are needed aside from the `additional_params` terms. Furthermore, for this tutorial, we use the network's default parameters when creating the `NetParams` object.

In [8]:
from flow.core.params import NetParams

net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

### 2.4 InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicle in the network at the start of a simulation. These parameters can be used to limit the edges and number of lanes vehicles originally occupy, and provide a means of adding randomness to the starting positions of vehicles. In order to introduce a small initial disturbance to the system of vehicles in the network, we set the `perturbation` term in `InitialConfig` to 1m.

In [9]:
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.5 TrafficLightParams

`TrafficLightParams` are used to describe the positions and types of traffic lights in the network. These inputs are outside the scope of this tutorial, and instead are covered in `tutorial10_traffic_lights.ipynb`. For our example, we create an empty `TrafficLightParams` object, thereby ensuring that none are placed on any nodes.

In [10]:
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams()

## 3. Setting up an Environment

Several envionrments in Flow exist to train autonomous agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. These environments are often network- or task-specific; however, some can be deployed on an ambiguous set of networks as well. One such environment, `AccelEnv`, may be used to train a variable number of vehicles in a fully observable network with a *static* number of vehicles.

In [11]:
from flow.envs.ring.accel import AccelEnv

Although we will not be training any autonomous agents in this tutorial, the use of an environment allows us to view the cumulative reward simulation rollouts receive in the absence of autonomy.

Although we will not be training any autonomous agents in this exercise, the use of an environment allows us to view the cumulative reward simulation rollouts receive in the absence of autonomy.

Envrionments in Flow are parametrized by several components, including the following attributes:
* `sim_params`
* `env_params`
* `network`
* `net_params`
* `initial_config`
* `network`
* `simulator`

where `sim_params`, `env_params`, and `network` are the primary parameters of an environment. For the full list of attributes, please check `class Env` in `flow/envs/base.py`.

Sumo envrionments in Flow are parametrized by three components:
* `SumoParams`
* `EnvParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables (e.g. `SumoParams` and `AimsunParams` are the variables related to SUMO and Aimsun simulator, respectively). These variables maay include the length a simulation step (in seconds), whether to render the GUI when running the experiment, and other variables. For this example, we consider a SUMO simulation, step length of 0.1s, and activate the GUI.

Another useful parameter is `emission_path`, which is used to specify the path where the emissions output will be generated. They contain a lot of information about the simulation, for instance the position and speed of each car at each time step. If you do not specify any emission path, the emission file will not be generated. More on this in Section 5.

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length a simulation step (in seconds) and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and activate the GUI.

Another useful parameter is `emission_path`, which is used to specify the path where the emissions output will be generated. They contain a lot of information about the simulation, for instance the position and speed of each car at each time step. If you do not specify any emission path, the emission file will not be generated. More on this in Section 5.

In [12]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

### 3.2 EnvParams

`EnvParams` specify environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. Much like `NetParams`, the attributes associated with this parameter are mostly environment-specific, and can be found in the environment's `ADDITIONAL_ENV_PARAMS` dictionary.

In [13]:
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS

print(ADDITIONAL_ENV_PARAMS)

{'max_accel': 3, 'max_decel': 3, 'target_velocity': 10, 'sort_vehicles': False}


Importing the `ADDITIONAL_ENV_PARAMS` variable, we see that it consists of only one entry, "target_velocity", which is used when computing the reward function associated with the environment. We use this default value when generating the `EnvParams` object.

In [14]:
from flow.core.params import EnvParams

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

## 4. Setting up and Running the Experiment
Once the inputs to the network and environment classes are ready, we are ready to set up a `Experiment` object.

In [15]:
from flow.core.experiment import Experiment

This object may be used to simulate rollouts in the absence of reinforcement learning agents, as well as acquire behaviors and rewards that may be used as a baseline with which to compare the performance of the learning agent. In this case, we choose to run our experiment for one rollout consisting of 3000 steps (300 s).

**Note**: When executing the below code, remeber to click on the    <img style="display:inline;" src="img/play_button.png"> Play button after the GUI is rendered.

In [16]:
flow_params = dict(
    exp_tag='ring_example',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# number of time steps
flow_params['env'].horizon = 3000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1, convert_to_csv=True)

Round 0, return: 883.8207358517438
data/ring_example_20210306-2106061615086366.735444-0_emission.csv data
Average, std returns: 883.8207358517438, 0.0
Average, std velocities: 2.9473562311491732, 0.0
Average, std outflows: 0.0, 0.0
Total time: 46.12213110923767
steps/second: 74.9168747288369


As we can see from the above simulation, the initial perturbations in the network instabilities propogate and intensify, eventually leading to the formation of stop-and-go waves after approximately 180s.

## 5. Visualizing Post-Simulation

Once the simulation is done, a .xml file will be generated in the location of the specified `emission_path` in `SumoParams` (assuming this parameter has been specified) under the name of the network. In our case, this is:

In [28]:
import os

emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.network.name)
print(emission_location + '-emission.xml')

data/ring_example_20210306-2106061615086366.735444-emission.xml


The .xml file contains various vehicle-specific parameters at every time step. This information is transferred to a .csv file if the `convert_to_csv` parameter in `exp.run()` is set to True. This file looks as follows:

In [31]:
import pandas as pd
import os
print(os.getcwd())
pd.read_csv(os.getcwd()+"/"+ emission_location + '-0_emission.csv')

/Users/Yulin_Zhang/Documents/gitlab/flow/tutorials


,time,id,x,y,speed,headway,leader_id,target_accel_with_noise_with_failsafe,target_accel_no_noise_no_failsafe,target_accel_with_noise_no_failsafe,target_accel_no_noise_with_failsafe,realized_accel,road_grade,edge_id,lane_number,distance,relative_position,follower_id,leader_rel_speed
0,0.0,human_0,36.640000,-1.600000,0.000000,4.990350,human_1,NaN,NaN,NaN,NaN,0.000000,0,bottom,0,0.000000,0.000000,human_21,0.000000
1,0.1,human_0,36.648663,-1.599828,0.083107,4.990388,human_1,0.839381,0.839381,0.839381,0.839381,0.831070,0,bottom,0,0.008311,0.008311,human_21,0.000387
2,0.2,human_0,36.665849,-1.599486,0.164865,4.990469,human_1,0.825760,0.825760,0.825760,0.825760,0.817584,0,bottom,0,0.024797,0.024797,human_21,0.000809
3,0.3,human_0,36.691413,-1.598979,0.245244,4.990598,human_1,0.811827,0.811827,0.811827,0.811827,0.803789,0,bottom,0,0.049322,0.049322,human_21,0.001282
4,0.4,human_0,36.725210,-1.598307,0.324216,4.990780,human_1,0.797618,0.797618,0.797618,0.797618,0.789721,0,bottom,0,0.081743,0.081743,human_21,0.001825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66017,299.6,human_21,-0.570331,27.849712,2.757112,5.248520,human_0,0.177290,0.177290,0.177290,0.177290,-0.010522,0,left,0,883.744298,8.450533,human_20,-0.000205
66018,299.7,human_21,-0.505442,27.569765,2.756233,5.248677,human_0,0.178339,0.178339,0.178339,0.178339,-0.008787,0,left,0,884.019922,8.726156,human_20,0.001564
66019,299.8,human_21,-0.435504,27.291144,2.755528,5.249009,human_0,0.179378,0.179378,0.179378,0.179378,-0.007052,0,left,0,884.295474,9.001709,human_20,0.003323
66020,299.9,human_21,-0.362311,27.013385,2.754996,5.249516,human_0,0.180408,0.180408,0.180408,0.180408,-0.005320,0,left,0,884.570974,9.277208,human_20,0.005070


As you can see, each row contains vehicle information for a certain vehicle (specified under the *id* column) at a certain time (specified under the *time* column). These information can then be used to plot various representations of the simulation, examples of which can be found in the `flow/visualize` folder.

## 6. Modifying the Simulation
This tutorial has walked you through running a single lane ring road experiment in Flow. As we have mentioned before, these simulations are highly parametrizable. This allows us to try different representations of the task. For example, what happens if no initial perturbations are introduced to the system of homogenous human-driven vehicles?

```
initial_config = InitialConfig()
```

In addition, how does the task change in the presence of multiple lanes where vehicles can overtake one another?

```
net_params = NetParams(
    additional_params={
        'length': 230, 
        'lanes': 2, 
        'speed_limit': 30, 
        'resolution': 40
    }
)
```

Feel free to experiment with all these problems and more!

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.